# V2: SEQ2SEQ

This notebook follows [an online tutorial](https://www.tensorflow.org/text/tutorials/nmt_with_attention#create_a_tfdata_dataset).

In [1]:
import tensorflow as tf

2024-09-04 21:20:07.244525: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-04 21:20:07.248411: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-04 21:20:07.258097: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-04 21:20:07.274324: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-04 21:20:07.278762: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-04 21:20:07.290596: I tensorflow/core/platform/cpu_feature_gu

**Notes**:
- This notebook follows [an online tutorial](https://www.tensorflow.org/text/tutorials/nmt_with_attention) (and [at least one other](https://www.tensorflow.org/text/tutorials/text_generation) of the Tensorflow tutorials).
- This [blog post](https://janakiev.com/blog/jupyter-virtual-envs/) was referenced to set up the virtual environment.

In [2]:
import numpy as np
from typing import Any, Tuple
from IPython.display import display, Markdown
from pathlib import Path


In [3]:
data_file_paths = list(Path('data/processed/en/').glob('*.txt'))
dataset_raw = tf.data.TextLineDataset(
	data_file_paths,
)

We've now loaded the `.txt` training data files using [`tf.data.TextLineDataset`](https://www.tensorflow.org/api_docs/python/tf/data/TextLineDataset). Each line in the source files is mapped to a new training example. 

Although some preprocessing has been done by `/data/process_data.py`, paragraphs aren't filtered out based on length/content. Let's do that now:

In [4]:
def filter_paragraphs(context, target):
	return tf.strings.length(context) > 5

punctuation_chars = r'\?!.,"\-\''
def add_context(target):
	context = tf.strings.regex_replace(target, '[{}]+'.format(punctuation_chars), '')
	context = tf.strings.strip(
		tf.strings.regex_replace(context, '[ ]+', ' ')
	)
	context = tf.strings.lower(context)
	return context, target

dataset_raw = dataset_raw.map(add_context).filter(filter_paragraphs)

Now let's inspect the data:

In [5]:
for text, label in dataset_raw.take(4).as_numpy_iterator():
	print(text, label)

b'illustration' b'Illustration '
b'alices adventures in wonderland' b"Alice's Adventures in Wonderland"
b'by lewis carroll' b'by Lewis Carroll'
b'the millennium fulcrum edition 30' b'THE MILLENNIUM FULCRUM EDITION 3.0'


2024-09-04 21:20:09.032851: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


### Batching

In [6]:
BUFFER_SIZE = 100_000
BATCH_SIZE = 64
dataset_train = dataset_raw.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

# Inspired by https://stackoverflow.com/a/74609848.
validate_size = 1
dataset_validate = dataset_train.take(validate_size)
dataset_train = dataset_train.skip(validate_size)

### Preparing to process data

The [`TextVectorization`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/TextVectorization) layer takes a `standardize` option that preprocesses input data. The default removes punctuation, but we don't want that. Let's redefine it:

In [7]:

def standardize_tf_text(text):
	punctuation_regex = '[{p}]+'.format(p = punctuation_chars)

	# Surround punctuation with spaces for easier tokenization
	text = tf.strings.regex_replace(text, punctuation_regex, r' \0 ')

	# Remove repeated spaces
	text = tf.strings.regex_replace(text, r'\s+', ' ')

	# Add a special "capitalize the next letter" token
	#text = tf.strings.regex_replace(text, r'(\s|^)([A-Z])', r' [CAP] \1')

	# Lowercase everything
	text = tf.strings.lower(text)

	# Add sequence markings
	text = tf.strings.strip(text)

	return tf.strings.join(['[START]', text, '[END]'], separator=' ')

The text standardization function can now be used to preprocess text:

In [8]:
# Keep only the 5000 most commonly used tokens
max_vocab_size = 5000

context_text_processor = tf.keras.layers.TextVectorization(
	standardize=standardize_tf_text,
	max_tokens=max_vocab_size,
	# Allow entries of different lengths
	ragged=True,
)
context_text_processor.adapt(dataset_train.map(lambda context, target: context))

print('First 14 context words:', context_text_processor.get_vocabulary()[:14])
print('Context vocab length', len(context_text_processor.get_vocabulary()))

target_text_processor = tf.keras.layers.TextVectorization(
	standardize=standardize_tf_text,
	max_tokens=max_vocab_size,
	ragged=True,
)
target_text_processor.adapt(dataset_train.map(lambda context, target: target))

print('First 14 target words:', target_text_processor.get_vocabulary()[:14])

2024-09-04 21:20:12.259210: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


First 14 context words: ['', '[UNK]', 'the', 'and', 'to', 'of', 'i', '[START]', '[END]', 'a', 'in', 'you', 'he', 'was']
Context vocab length 5000
First 14 target words: ['', '[UNK]', '[cap]', ',', 'the', '.', 'and', 'to', 'of', '"', '[START]', '[END]', 'a', 'he']


We can use these layers to convert to/from token IDs:


In [9]:
example_text = 'hello world this is a test tensorflow is processing this'
example_tokens = context_text_processor(example_text)
print('Example tokens', example_tokens)

context_vocab = np.array(context_text_processor.get_vocabulary())
tokens = context_vocab[example_tokens.numpy()]
print('Back to text', ' '.join(tokens))

Example tokens tf.Tensor([   7    1  264   35   20    9 4174    1   20    1   35    8], shape=(12,), dtype=int64)
Back to text [START] [UNK] world this is a test [UNK] is [UNK] this [END]


### Processing the data

Now, we'll:
1. Map the data through the text processors we just made.
2. Shift the target data, so that our network is provided with a history of generated tokens.

In [10]:
def process_text(context, target):
	return context_text_processor(context), target_text_processor(target)

def add_target_history(context, target):
	# .to_tensor(): Converts from RaggedTensors to Tensors.
	# We give our network the history as target_in
	target_in = target[:, :-1].to_tensor()
	target_out = target[:, 1:].to_tensor()
	return (context.to_tensor(), target_in), target_out
dataset_train = dataset_train.map(process_text).map(add_target_history)
dataset_validate = dataset_validate.map(process_text).map(add_target_history)

## Model

### The encoder

See https://www.tensorflow.org/text/tutorials/nmt_with_attention#the_encoder

In [11]:
class Encoder(tf.keras.Layer):
	def __init__(self, text_processor, units: int):
		"""
		Creates a new Encoder layer. [dimen] is the maxiumum number of elements of the input
		that can be processed by the encoder.
		"""
		super(Encoder, self).__init__()
		self.text_processor = text_processor
		self.vocab_size = text_processor.vocabulary_size()
		self.units = units

		# Converts tokens -> vectors
		self.embedding = tf.keras.layers.Embedding(
			# mask_zero: Treats zero as a padding value that should be ignored
			self.vocab_size, units, mask_zero = True,
		)
		gru = tf.keras.layers.GRU(
			units, return_sequences = True,
			# Use the recurrent_initializer suggested by the tutorial (& the default
			# for kernel_initializer).
			recurrent_initializer='glorot_uniform'
		)
		self.rnn = tf.keras.layers.Bidirectional(
			# merge_mode determines how the forward and backward layers are combined
			#            'concat' is another option here
			merge_mode = 'sum',
			layer=gru,
		)

	def call(self, x):
		x = self.embedding(x)
		x = self.rnn(x)
		return x

	def convert_input(self, texts):
		# TODO: Can this be removed? It doesn't seem to be documented...
		print('convert called!')
		texts = tf.convert_to_tensor(texts)
		if len(texts.shape) == 0:
			texts = texts[None]
		context = self.text_processor(texts).to_tensor()
		context = self(context)
		return context


Try it:

In [12]:
ENCODER_UNITS = 256
encoder = Encoder(context_text_processor, ENCODER_UNITS)

for (context, target_history), target_next in dataset_validate.take(1):
	encoder_result = encoder(context)
	print('Context tokens shape (batch, s):', context.shape)
	print('Encoder output shape (batch, s, ENCODER_UNITS):', encoder_result.shape)

Context tokens shape (batch, s): (64, 463)
Encoder output shape (batch, s, ENCODER_UNITS): (64, 463, 256)


2024-09-04 21:20:22.118538: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


### The attention layer

Attention can be thought of as training a lookup table with keys and values. The lookup table has inputs `values` and `query`.

In [13]:
class CrossAttention(tf.keras.Layer):
	def __init__(self, units, **kwargs):
		super().__init__()
		self.attention_layer = tf.keras.layers.MultiHeadAttention(
			key_dim=units,
			num_heads=1,
			**kwargs
		)
		# Keeps "the mean activation within each example close to 0 and the
		# activation standard deviation close to 1" -- https://www.tensorflow.org/api_docs/python/tf/keras/layers/LayerNormalization?hl=en
		self.norm_layer = tf.keras.layers.LayerNormalization()
		self.add_layer = tf.keras.layers.Add()
		self.supports_masking = True

	def call(self, query, value):
		attention_output = self.attention_layer(
			query = query,
			value = value,
			use_causal_mask=True,
			# Return the attention scores for latter plotting
			# return_attention_scores = True,
		)

		x = self.add_layer([ query, attention_output ])
		x = self.norm_layer(x)
		return x


In [14]:
attention_layer = CrossAttention(ENCODER_UNITS)

# Test with an example
for (context, target_history), target_next in dataset_validate.take(1):
	embed_layer = tf.keras.layers.Embedding(target_text_processor.vocabulary_size(), output_dim=ENCODER_UNITS, mask_zero=True)
	target_embed = embed_layer(target_history)
	encoded_context = encoder(context)
	attention_result = attention_layer(target_embed, encoded_context)

	print('Encoded context sequence shape (batch, s, units):', encoded_context.shape)
	print('Target history sequence shape (batch, t, units):', target_embed.shape)
	print('Attention result shape (batch, t, units):', attention_result.shape)

	# Used later 
	test_encoded_context = encoded_context

/home/self/Documents/punctuation-fixer/env/lib64/python3.11/site-packages/keras/src/layers/layer.py:934: UserWarning: Layer 'query' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/home/self/Documents/punctuation-fixer/env/lib64/python3.11/site-packages/keras/src/layers/layer.py:934: UserWarning: Layer 'key' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/home/self/Documents/punctuation-fixer/env/lib64/python3.11/site-packages/keras/src/layers/layer.py:934: UserWarning: Layer 'value' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information

Encoded context sequence shape (batch, s, units): (64, 123, 256)
Target history sequence shape (batch, t, units): (64, 159, 256)
Attention result shape (batch, t, units): (64, 159, 256)


### The decoder

The decoder produces queries for the attention layer. The decoder operates on `target_history`. At each step during training, it should have no information about future target output (that's what we're trying to determine). As such, we use a unidirectional RNN.


In [15]:
class CustomDense(tf.keras.layers.Dense):
	def __init__(self, *args, **kwargs):
		super(CustomDense, self).__init__(*args, **kwargs)
	
	def compute_mask(self, _inputs, mask=None):
		return mask

class Decoder(tf.keras.Layer):
	def __init__(self, text_processor, units):
		super(Decoder, self).__init__()
		self.text_processor = text_processor
		self.vocab_size = text_processor.vocabulary_size()
		self.units = units

		self.embedding_layer = tf.keras.layers.Embedding(
			# mask_zero: Treats zero as a padding value that should be ignored
			self.vocab_size, units, mask_zero = True,
		)
		self.rnn_layer = tf.keras.layers.GRU(
			units, return_sequences = True, return_state = True, recurrent_initializer='glorot_uniform',
		)
		self.attention_layer = CrossAttention(units)

		# Creates logits with the estimated probability of each output token
		self.output_layer = CustomDense(self.vocab_size)

		# Conversion:
		self.word_to_id = tf.keras.layers.StringLookup(
			vocabulary = text_processor.get_vocabulary(),
			mask_token = '',
			oov_token = '[UNK]',
		)
		self.id_to_word = tf.keras.layers.StringLookup(
			vocabulary = text_processor.get_vocabulary(),
			mask_token = '',
			oov_token = '[UNK]',
			invert = True,
		)
		self.supports_masking = True
	
	def build(self, input_shape):
		# Nothing tha needs a size allocation based on the input shape
		pass
	
	def call(self, context, target_history, state = None, return_state = False):
		x = self.embedding_layer(target_history)
		x, state = self.rnn_layer(x)
		x = self.attention_layer(x, context)

		logits = self.output_layer(x)
		if return_state:
			return logits, state
		else:
			return logits
	
	## Conversion/testing ##

	def tokens_to_text(self, tokens):
		return tf.strings.reduce_join(self.id_to_word(tokens), separator = ' ')

	def generate_next_token(self, context, target_history, done_vec, state, temperature = 0.0):
		# Note: is_done is a vector, indicating whether each item in the batch is done

		logits, state = self(context, target_history, state = state, return_state = True)

		# logits has shape (batch, t, target_vocab_size). Only generate the token corresponding
		# to the last logits in the sequence (at t - 1)
		if temperature > 0:
			next_token = tf.where(
				done_vec,
				tf.constant(0, dtype=tf.int64), # Emit 0 after a sequence is done
				tf.random.categorical(logits[:, -1, :] / temperature, num_samples = 1), # Otherwise, pick the token from a categorical distribution
			)
		else:
			next_token = tf.math.argmax(logits, axis=-1)
		done_vec = done_vec|(next_token == self.word_to_id('[END]'))
		return next_token, done_vec, state
	
	def get_initial_state(self, context):
		# context has shape (batch_size, s, units)
		batch_size = context.shape[0]
		start_tokens = tf.fill([batch_size, 1], self.word_to_id('[START]'))
		done_vec = tf.zeros([batch_size, 1], dtype = tf.bool)

		# From the Tensorflow source code:
		# > RNN expect the states in a list, even if single state.
		initial_state = self.rnn_layer.get_initial_state(batch_size)[0]

		return start_tokens, done_vec, initial_state

Let's try it!

In [16]:
def test_generation_loop():
	decoder = Decoder(target_text_processor, ENCODER_UNITS)
	next_token, done_vec, state = decoder.get_initial_state(test_encoded_context[:3, :, :])
	tokens = [next_token]

	for i in range(8):
		next_token, done_vec, state = decoder.generate_next_token(test_encoded_context[:3, :, :], next_token, done_vec, state)
		tokens.append(next_token)
	
	# Merge all batch outputs into a single dimension
	tokens = tf.concat(tokens, -1) # -1 = last axis

	print('Output:', decoder.tokens_to_text(tokens).numpy())

test_generation_loop()

Output: b'[START] wisdom aris opolo shuddered drawer witness oman studies [START] wisdom aris opolo shuddered drawer witness oman studies [START] wisdom aris opolo shuddered drawer witness oman studies'


## The model

We can now build a model for training and punctuation:

In [17]:
class Punctuator(tf.keras.Model):
	def __init__(self, units, context_text_processor, target_text_processor):
		super().__init__()
		self.encoder = Encoder(context_text_processor, units)
		self.decoder = Decoder(target_text_processor, units)
	
	def call(self, inputs):
		context, target_history = inputs
		context = self.encoder(context)
		logits = self.decoder(context, target_history)
		return logits

In [18]:
model = Punctuator(ENCODER_UNITS, context_text_processor, target_text_processor)

for (example_context_tok, example_target_hist), _ in dataset_validate.take(1):
	test_logits = model((example_context_tok, example_target_hist))
	print('Context tokens shape (batch, s, units):', example_context_tok.shape)
	print('Target history tokens shape (batch, s, units):', example_target_hist.shape)
	print('Logits shape (batch, t, vocab_size)', test_logits.shape)

Context tokens shape (batch, s, units): (64, 278)
Target history tokens shape (batch, s, units): (64, 351)
Logits shape (batch, t, vocab_size) (64, 351, 5000)


To avoid penalizing masked outputs, we use a custom loss function (see the tutorial):

In [19]:
base_loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

def masked_loss(y_true, y_predict):
	loss = base_loss_fn(y_true, y_predict)
	
	unmasked = y_true != 0
	unmasked = tf.cast(unmasked, loss.dtype)
	# Only consider output with a corresponding label.
	loss *= unmasked

	count_unmasked = tf.math.reduce_sum(unmasked)

	# reduce_sum: Adds all entries of a vector.
	return tf.math.reduce_sum(loss)/count_unmasked

In [20]:
def masked_accuracy(y_true, predict_logits):
	predicted_index = tf.math.argmax(predict_logits, axis=-1)
	predicted_index = tf.cast(predicted_index, y_true.dtype)

	match = tf.cast(y_true == predicted_index, tf.float32)
	unmasked = tf.cast(y_true != 0, tf.float32)
	count_unmasked = tf.math.reduce_sum(unmasked)

	return tf.math.reduce_sum(match * unmasked) / count_unmasked


In [21]:
model.compile(optimizer='adam', loss=masked_loss, metrics=[masked_accuracy, masked_loss])

In [22]:
print('From the tutorial:')
vocab_size = float(target_text_processor.vocabulary_size())

print('expected loss', tf.math.log(vocab_size).numpy())
print('expected accuracy', 1/vocab_size)

From the tutorial:
expected loss 8.517193
expected accuracy 0.0002


In [23]:
model.evaluate(dataset_validate, steps=20, return_dict=True)


20/20 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 8.5275 - masked_accuracy: 0.0000e+00 - masked_loss: 4.4769 


/usr/lib64/python3.11/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


{'loss': 8.527496337890625,
 'masked_accuracy': 0.0,
 'masked_loss': 4.263747215270996}

In [25]:
history = model.fit(
	dataset_train,
	epochs = 10,
	steps_per_epoch = 100,
	validation_data = dataset_validate
)

Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 314s 3s/step - loss: 4.7753 - masked_accuracy: 0.2177 - masked_loss: 4.7753 - val_loss: 4.0922 - val_masked_accuracy: 0.1540 - val_masked_loss: 2.0461
Epoch 2/10


2024-09-04 21:28:20.937660: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/usr/lib64/python3.11/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


100/100 ━━━━━━━━━━━━━━━━━━━━ 332s 3s/step - loss: 4.0356 - masked_accuracy: 0.3029 - masked_loss: 4.0356 - val_loss: 3.6247 - val_masked_accuracy: 0.1776 - val_masked_loss: 1.8124
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 339s 3s/step - loss: 2.9644 - masked_accuracy: 0.4739 - masked_loss: 2.9644 - val_loss: 1.4176 - val_masked_accuracy: 0.3619 - val_masked_loss: 0.7088
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 309s 3s/step - loss: 1.3654 - masked_accuracy: 0.7389 - masked_loss: 1.3654 - val_loss: 0.7307 - val_masked_accuracy: 0.4208 - val_masked_loss: 0.3654
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 245s 2s/step - loss: 0.8032 - masked_accuracy: 0.8267 - masked_loss: 0.8014 - val_loss: 0.8403 - val_masked_accuracy: 0.4118 - val_masked_loss: 0.4201
Epoch 6/10
  7/100 ━━━━━━━━━━━━━━━━━━━━ 4:14 3s/step - loss: 0.6504 - masked_accuracy: 0.8525 - masked_loss: 0.6504

KeyboardInterrupt: 

In [38]:
def test_punctuation(text):
	context = model.encoder.convert_input(text)
	print(context.shape)

	tokens = []
	next_token, done_vec, state = model.decoder.get_initial_state(context)
	tokens = [next_token]

	for i in range(100):
		next_token, done_vec, state = model.decoder.generate_next_token(context, next_token, done_vec, state, temperature=1.0)
		tokens.append(next_token)
		if tf.reduce_all(done_vec):
			break
	
	# Merge all batch outputs into a single dimension
	tokens = tf.concat(tokens, -1) # -1 = last axis

	print('Output:', model.decoder.tokens_to_text(tokens).numpy())

test_punctuation([ 'this is a test of a thing' ])

convert called!
(1, 9, 256)


/home/self/Documents/punctuation-fixer/env/lib64/python3.11/site-packages/keras/src/layers/layer.py:934: UserWarning: Layer 'query' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/home/self/Documents/punctuation-fixer/env/lib64/python3.11/site-packages/keras/src/layers/layer.py:934: UserWarning: Layer 'key' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/home/self/Documents/punctuation-fixer/env/lib64/python3.11/site-packages/keras/src/layers/layer.py:934: UserWarning: Layer 'value' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information

Output: b'[START] " shadow in this out where an idea in this better interrupted of ship still at her confidence in fifty turned it in considerable hand in this tongue in this head in [cap] ead near my cup in placing my confidence in his brain in this seat in [END]'
